In [17]:
import pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture as GMM
from FeatureExtraction import extract_features
#from speakerfeatures import extract_features
import warnings
import os
import sys
import time
import _pickle as cPickle
#from speakerfeatures import extract_features
warnings.filterwarnings("ignore")
import glob


warnings.filterwarnings("ignore")

time1 = time.time()
training = True
dest = "4-gmm_models/"

with open("training_size.txt", "w") as f:
    print("training size (5-second utterances)\tTraining Time - classes (seconds)\taccuracy testing_same\tTesting Time per Sample (ms)\taccuracy testing_different\tTesting Time per Sample (ms)", file=f)


training_size = 30

test = "normalized"

subset = "training"
audios = f"../../dataset/{test}_dataset/{subset}/"

directories = os.listdir(audios)
print(directories)



for directory in directories:
    # 1 speaker
    # directory = directories[0]

    # List all files in the directory
    file_paths = os.listdir(audios + directory)
    file_paths = sorted(file_paths)
    # print(file_paths)
    # Extracting features for each speaker
    features = np.asarray(())
    for path in file_paths:
        path = directory + "/" + path    
        path = path.strip()   
        # print (path)
        
        # read the audio
        sr,audio = read(audios+path)
        
        # extract 40 dimensional MFCC & delta MFCC features
        vector   = extract_features(audio,sr)
        
        if features.size == 0:
            features = vector
        else:
            features = np.vstack((features, vector))
        # print(features.shape)

        

    if training and subset == "training":            
        gmm = GMM(n_components = 5, covariance_type='diag',n_init = 3)
        gmm.fit(features)
        # dumping the trained gaussian model
        picklefile = directory + ".gmm"
        print(picklefile)
        cPickle.dump(gmm,open(dest + picklefile,'wb'))
        # print ('+ modeling completed for speaker:',picklefile," with data point = ",features.shape)   
        features = np.asarray(())
print("Total Time taken: ", round(time.time() - time1, 2), "seconds")
with open("training_size.txt", "a") as f:
    print(f"{training_size}\t{round(time.time() - time1, 2)}", file=f, end="\t")


['Sherif', 'Youssef', 'Abdelrahman', 'Omar', 'Reem', 'Renad']
Sherif.gmm
Youssef.gmm
Abdelrahman.gmm
Omar.gmm
Reem.gmm
Renad.gmm
Total Time taken:  5.6 seconds


In [18]:
test = "normalized"
subset = "testing_same"

# test = "normalized"
# subset = "testing_different"

source = f"../../dataset/{test}_dataset/{subset}/"

#path where training speakers will be saved
modelpath = "4-gmm_models"

gmm_files = [os.path.join(modelpath,fname) for fname in 
              os.listdir(modelpath) if fname.endswith('.gmm')]

#Load the Gaussian Models
models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
print("Loaded Models: ", len(models))
speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname 
              in gmm_files]

error = 0
total_samples = 0.0


predictions = []
gender_predictions = []
gender_gt = []
y_true = []
preprocessingMisPredictions = 0
print("Testing the model with all the samples...")
false_predictions = []
test_file = "Testing_audio_Path.txt"       
# file_paths = open(test_file,'r')
file_paths = glob.glob(f"{source}/**/*.wav", recursive=True)

# Read the test directory and get the list of test audio files 
avg_time = 0
total_time = 0
for path in file_paths:
    time1 = time.time()   
    total_samples+= 1.0
    path=path.strip()
    # print("Testing Audio : ", path)
    sr,audio = read(path)
    vector   = extract_features(audio,sr)
    # print(vector.shape)
    log_likelihood = np.zeros(len(models)) 
    for i in range(len(models)):
        gmm    = models[i]  #checking with each model one by one
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
    winner=np.argmax(log_likelihood)
    winner_score = log_likelihood[winner]
    predicted_speaker = speakers[winner]
    speaker_label = path.split("/")[-2]
    # print("OK score: ", winner_score)
    predictions.append(predicted_speaker)
    y_true.append(speaker_label)
    if speaker_label == "Abdelrahman" or speaker_label == "Youssef" or speaker_label == "Sherif" or speaker_label == "Omar":
        gender_predictions.append("M")
    else:
        gender_predictions.append("F")
    if predicted_speaker == "Abdelrahman" or predicted_speaker == "Youssef" or predicted_speaker == "Sherif" or predicted_speaker == "Omar":
        gender_gt.append("M")
    else:
        gender_gt.append("F")
    if predicted_speaker != speaker_label:
        print(f"{speaker_label}:{predicted_speaker}, ", end="")
        # print(f"False Score: {winner_score}")
        # print(path)
        error += 1
    # time.sleep(1.0)
    sample_time = time.time() - time1
    total_time += sample_time
print (error, total_samples)
accuracy = ((total_samples - error) / total_samples)

print ("Accuracy: ", round(accuracy, 4))
print(f"Average time taken per sample in ms", round((total_time/total_samples)*1000, 2))
print("Preprocessing MisPredictions: ", preprocessingMisPredictions)
# print ("The following Predictions were False :")
# print (false_predictions)
print(modelpath)
print ("Speaker Identified Successfully")
with open("training_size.txt", "a") as f:
    print(f"{round(accuracy, 4)}\t{round((total_time/total_samples)*1000, 2)}", file=f, end="\t")



Loaded Models:  6
Testing the model with all the samples...
Reem:Renad, 1 120.0
Accuracy:  0.9917
Average time taken per sample in ms 15.22
Preprocessing MisPredictions:  0
4-gmm_models
Speaker Identified Successfully


In [19]:
test = "normalized"
subset = "testing_different"

# test = "normalized"
# subset = "testing_different"

source = f"../../dataset/{test}_dataset/{subset}/"

#path where training speakers will be saved
modelpath = "4-gmm_models"

gmm_files = [os.path.join(modelpath,fname) for fname in 
              os.listdir(modelpath) if fname.endswith('.gmm')]

#Load the Gaussian Models
models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
print("Loaded Models: ", len(models))
speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname 
              in gmm_files]

error = 0
total_samples = 0.0


predictions = []
gender_predictions = []
gender_gt = []
y_true = []
preprocessingMisPredictions = 0
print("Testing the model with all the samples...")
false_predictions = []
test_file = "Testing_audio_Path.txt"       
# file_paths = open(test_file,'r')
file_paths = glob.glob(f"{source}/**/*.wav", recursive=True)

# Read the test directory and get the list of test audio files 
avg_time = 0
total_time = 0
for path in file_paths:
    time1 = time.time()   
    total_samples+= 1.0
    path=path.strip()
    # print("Testing Audio : ", path)
    sr,audio = read(path)
    vector   = extract_features(audio,sr)
    # print(vector.shape)
    log_likelihood = np.zeros(len(models)) 
    for i in range(len(models)):
        gmm    = models[i]  #checking with each model one by one
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
    winner=np.argmax(log_likelihood)
    winner_score = log_likelihood[winner]
    predicted_speaker = speakers[winner]
    speaker_label = path.split("/")[-2]
    # print("OK score: ", winner_score)
    predictions.append(predicted_speaker)
    y_true.append(speaker_label)
    if speaker_label == "Abdelrahman" or speaker_label == "Youssef" or speaker_label == "Sherif" or speaker_label == "Omar":
        gender_predictions.append("M")
    else:
        gender_predictions.append("F")
    if predicted_speaker == "Abdelrahman" or predicted_speaker == "Youssef" or predicted_speaker == "Sherif" or predicted_speaker == "Omar":
        gender_gt.append("M")
    else:
        gender_gt.append("F")
    if predicted_speaker != speaker_label:
        print(f"{speaker_label}:{predicted_speaker}, ", end="")
        # print(f"False Score: {winner_score}")
        # print(path)
        error += 1
    # time.sleep(1.0)
    sample_time = time.time() - time1
    total_time += sample_time
print (error, total_samples)
accuracy = ((total_samples - error) / total_samples)

print ("Accuracy: ", round(accuracy, 4))
print(f"Average time taken per sample in ms", round((total_time/total_samples)*1000, 2))
print("Preprocessing MisPredictions: ", preprocessingMisPredictions)
# print ("The following Predictions were False :")
# print (false_predictions)
print(modelpath)
print ("Speaker Identified Successfully")
with open("training_size.txt", "a") as f:
    print(f"{round(accuracy, 4)}\t{round((total_time/total_samples)*1000, 2)}", file=f, end="\t")



Loaded Models:  6
Testing the model with all the samples...
Sherif:Reem, Sherif:Omar, Sherif:Reem, Sherif:Reem, Sherif:Renad, Youssef:Abdelrahman, Youssef:Abdelrahman, Youssef:Reem, Youssef:Sherif, Youssef:Abdelrahman, Abdelrahman:Omar, Abdelrahman:Reem, Omar:Sherif, Omar:Sherif, Reem:Renad, Reem:Renad, Reem:Sherif, Reem:Renad, Renad:Reem, 19 48.0
Accuracy:  0.6042
Average time taken per sample in ms 15.0
Preprocessing MisPredictions:  0
4-gmm_models
Speaker Identified Successfully


In [21]:
import pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture as GMM
from FeatureExtraction import extract_features
#from speakerfeatures import extract_features
import warnings
import os
import sys
import time
import _pickle as cPickle
#from speakerfeatures import extract_features
warnings.filterwarnings("ignore")
import glob


warnings.filterwarnings("ignore")

time1 = time.time()
training = True
dest = "4-gmm_models/"

with open("training_size.txt", "w") as f:
    print("training size (5-second utterances)\tTraining Time - classes (seconds)\taccuracy testing_same\tTesting Time per Sample (ms)\taccuracy testing_different\tTesting Time per Sample (ms)", file=f)

training_size = 31
while training_size > 1:
    training_size -= 1
    test = "normalized"

    subset = "training"
    audios = f"../../dataset/{test}_dataset/{subset}/"

    directories = os.listdir(audios)
    print(directories)



    for directory in directories:
        # 1 speaker
        # directory = directories[0]

        # List all files in the directory
        file_paths = os.listdir(audios + directory)
        file_paths = sorted(file_paths)
        file_paths = file_paths[:training_size]
        # print(file_paths)
        # Extracting features for each speaker
        features = np.asarray(())
        for path in file_paths:
            path = directory + "/" + path    
            path = path.strip()   
            # print (path)
            
            # read the audio
            sr,audio = read(audios+path)
            
            # extract 40 dimensional MFCC & delta MFCC features
            vector   = extract_features(audio,sr)
            
            if features.size == 0:
                features = vector
            else:
                features = np.vstack((features, vector))
            # print(features.shape)

            

        if training and subset == "training":            
            gmm = GMM(n_components = 5, covariance_type='diag',n_init = 3)
            gmm.fit(features)
            # dumping the trained gaussian model
            picklefile = directory + ".gmm"
            print(picklefile)
            cPickle.dump(gmm,open(dest + picklefile,'wb'))
            # print ('+ modeling completed for speaker:',picklefile," with data point = ",features.shape)   
            features = np.asarray(())
    print("Total Time taken: ", round(time.time() - time1, 2), "seconds")
    with open("training_size.txt", "a") as f:
        print(f"{training_size}\t{round(time.time() - time1, 2)}", file=f, end="\t")

    test = "normalized"
    subset = "testing_same"

    # test = "normalized"
    # subset = "testing_different"

    source = f"../../dataset/{test}_dataset/{subset}/"

    #path where training speakers will be saved
    modelpath = "4-gmm_models"

    gmm_files = [os.path.join(modelpath,fname) for fname in 
                os.listdir(modelpath) if fname.endswith('.gmm')]

    #Load the Gaussian Models
    models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
    print("Loaded Models: ", len(models))
    speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname 
                in gmm_files]

    error = 0
    total_samples = 0.0


    predictions = []
    gender_predictions = []
    gender_gt = []
    y_true = []
    preprocessingMisPredictions = 0
    print("Testing the model with all the samples...")
    false_predictions = []
    test_file = "Testing_audio_Path.txt"       
    # file_paths = open(test_file,'r')
    file_paths = glob.glob(f"{source}/**/*.wav", recursive=True)

    # Read the test directory and get the list of test audio files 
    avg_time = 0
    total_time = 0
    for path in file_paths:
        time1 = time.time()   
        total_samples+= 1.0
        path=path.strip()
        # print("Testing Audio : ", path)
        sr,audio = read(path)
        vector   = extract_features(audio,sr)
        # print(vector.shape)
        log_likelihood = np.zeros(len(models)) 
        for i in range(len(models)):
            gmm    = models[i]  #checking with each model one by one
            scores = np.array(gmm.score(vector))
            log_likelihood[i] = scores.sum()
        winner=np.argmax(log_likelihood)
        winner_score = log_likelihood[winner]
        predicted_speaker = speakers[winner]
        speaker_label = path.split("/")[-2]
        # print("OK score: ", winner_score)
        predictions.append(predicted_speaker)
        y_true.append(speaker_label)
        if speaker_label == "Abdelrahman" or speaker_label == "Youssef" or speaker_label == "Sherif" or speaker_label == "Omar":
            gender_predictions.append("M")
        else:
            gender_predictions.append("F")
        if predicted_speaker == "Abdelrahman" or predicted_speaker == "Youssef" or predicted_speaker == "Sherif" or predicted_speaker == "Omar":
            gender_gt.append("M")
        else:
            gender_gt.append("F")
        if predicted_speaker != speaker_label:
            print(f"{speaker_label}:{predicted_speaker}, ", end="")
            # print(f"False Score: {winner_score}")
            # print(path)
            error += 1
        # time.sleep(1.0)
        sample_time = time.time() - time1
        total_time += sample_time
    print (error, total_samples)
    accuracy = ((total_samples - error) / total_samples)

    print ("Accuracy: ", round(accuracy, 4))
    print(f"Average time taken per sample in ms", round((total_time/total_samples)*1000, 2))
    print("Preprocessing MisPredictions: ", preprocessingMisPredictions)
    # print ("The following Predictions were False :")
    # print (false_predictions)
    print(modelpath)
    print ("Speaker Identified Successfully")
    with open("training_size.txt", "a") as f:
        print(f"{round(accuracy, 4)}\t{round((total_time/total_samples)*1000, 2)}", file=f, end="\t")


    test = "normalized"
    subset = "testing_different"

    # test = "normalized"
    # subset = "testing_different"

    source = f"../../dataset/{test}_dataset/{subset}/"

    #path where training speakers will be saved
    modelpath = "4-gmm_models"

    gmm_files = [os.path.join(modelpath,fname) for fname in 
                os.listdir(modelpath) if fname.endswith('.gmm')]

    #Load the Gaussian Models
    models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
    print("Loaded Models: ", len(models))
    speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname 
                in gmm_files]

    error = 0
    total_samples = 0.0


    predictions = []
    gender_predictions = []
    gender_gt = []
    y_true = []
    preprocessingMisPredictions = 0
    print("Testing the model with all the samples...")
    false_predictions = []
    test_file = "Testing_audio_Path.txt"       
    # file_paths = open(test_file,'r')
    file_paths = glob.glob(f"{source}/**/*.wav", recursive=True)

    # Read the test directory and get the list of test audio files 
    avg_time = 0
    total_time = 0
    for path in file_paths:
        time1 = time.time()   
        total_samples+= 1.0
        path=path.strip()
        # print("Testing Audio : ", path)
        sr,audio = read(path)
        vector   = extract_features(audio,sr)
        # print(vector.shape)
        log_likelihood = np.zeros(len(models)) 
        for i in range(len(models)):
            gmm    = models[i]  #checking with each model one by one
            scores = np.array(gmm.score(vector))
            log_likelihood[i] = scores.sum()
        winner=np.argmax(log_likelihood)
        winner_score = log_likelihood[winner]
        predicted_speaker = speakers[winner]
        speaker_label = path.split("/")[-2]
        # print("OK score: ", winner_score)
        predictions.append(predicted_speaker)
        y_true.append(speaker_label)
        if speaker_label == "Abdelrahman" or speaker_label == "Youssef" or speaker_label == "Sherif" or speaker_label == "Omar":
            gender_predictions.append("M")
        else:
            gender_predictions.append("F")
        if predicted_speaker == "Abdelrahman" or predicted_speaker == "Youssef" or predicted_speaker == "Sherif" or predicted_speaker == "Omar":
            gender_gt.append("M")
        else:
            gender_gt.append("F")
        if predicted_speaker != speaker_label:
            print(f"{speaker_label}:{predicted_speaker}, ", end="")
            # print(f"False Score: {winner_score}")
            # print(path)
            error += 1
        # time.sleep(1.0)
        sample_time = time.time() - time1
        total_time += sample_time
    print (error, total_samples)
    accuracy = ((total_samples - error) / total_samples)

    print ("Accuracy: ", round(accuracy, 4))
    print(f"Average time taken per sample in ms", round((total_time/total_samples)*1000, 2))
    print("Preprocessing MisPredictions: ", preprocessingMisPredictions)
    # print ("The following Predictions were False :")
    # print (false_predictions)
    print(modelpath)
    print ("Speaker Identified Successfully")
    with open("training_size.txt", "a") as f:
        print(f"{round(accuracy, 4)}\t{round((total_time/total_samples)*1000, 2)}", file=f)



['Sherif', 'Youssef', 'Abdelrahman', 'Omar', 'Reem', 'Renad']
Sherif.gmm
Youssef.gmm
Abdelrahman.gmm
Omar.gmm
Reem.gmm
Renad.gmm
Total Time taken:  6.13 seconds
Loaded Models:  6
Testing the model with all the samples...
0 120.0
Accuracy:  1.0
Average time taken per sample in ms 21.53
Preprocessing MisPredictions:  0
4-gmm_models
Speaker Identified Successfully
Loaded Models:  6
Testing the model with all the samples...
Sherif:Reem, Sherif:Reem, Sherif:Renad, Youssef:Abdelrahman, Youssef:Abdelrahman, Youssef:Renad, Youssef:Sherif, Youssef:Abdelrahman, Abdelrahman:Omar, Omar:Sherif, Omar:Youssef, Omar:Abdelrahman, Omar:Abdelrahman, Omar:Sherif, Omar:Youssef, Reem:Renad, Reem:Renad, Reem:Sherif, Reem:Renad, 19 48.0
Accuracy:  0.6042
Average time taken per sample in ms 21.28
Preprocessing MisPredictions:  0
4-gmm_models
Speaker Identified Successfully
['Sherif', 'Youssef', 'Abdelrahman', 'Omar', 'Reem', 'Renad']
Sherif.gmm
Youssef.gmm
Abdelrahman.gmm
Omar.gmm
Reem.gmm
Renad.gmm
Total Time

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
#previous error is just due to an iteration of size 0. just keep it while > 1 and it works anyway